In [ ]:
import carla, time, pygame, math, random, cv2
from carla import Sensor
import math
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.mercedes.coupe_2020', rotation=None):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    if rotation:
        spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle


def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x = 2.0, z=1.0)), horizontal_fov=10, range=100, vertical_fov=10, points_per_second=1500, sensor_tick=0.0):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('range', str(range))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar_bp.set_attribute('sensor_tick', str(sensor_tick))
    radar = world.spawn_actor(radar_bp, transform=transform, attach_to=attach_to)
    return radar

def remove_all(world):
    for a in world.get_actors().filter('vehicle.*'):
        a.destroy()
    for a in world.get_actors().filter('sensor.*'):
        a.destroy()




In [29]:
remove_all(world)

vehicle = spawn_vehicle(rotation=carla.Rotation(yaw=70.0))
camera = spawn_camera(attach_to=vehicle, transform=carla.Transform(
        carla.Location(x=0.3, y=0.0, z=1.5),  # Offset from vehicle's center: forward, center, up
        carla.Rotation(pitch=-10.0)  # Looking backward
    ) )

video_output = np.zeros((600, 800, 4), dtype=np.uint8)

def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

camera.listen(lambda image: camera_callback(image))


cv2.namedWindow('Front Camera', cv2.WINDOW_AUTOSIZE)


running = True
try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('Front Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()


